# Focus stacking with image bunches

Example of focus stacking with automatic alignment and luminosity balance, and partial stacking of image bunches before the final stacking.

In [2]:
from focus_stack import *

In [4]:
def make_job(x, y):
    job = StackJob("job", x + y, input_path="src")
    job.add_action(MultiRefActions("align", actions=[AlignLayers(),
                                                     Balance(LumiCorrection(i_min=125, i_max=65385, mask_size=0.8))]))
    job.add_action(FocusStackBunch("bunches", PyramidStack(), exif_path="src", frames=13, overlap=2, denoise=0))
    job.add_action(FocusStack("stack", PyramidStack(), exif_path="src", postfix='_pyr', denoise=0))
    return job

In [5]:
for x in ["E:/Focus stacking/2025-04-21 - Hemiptera/"]:
    for y in ['J']:
        make_job(x, y).run()

88 files in folder: 'E:/Focus stacking/2025-04-21 - Hemiptera/J/src'


align:   0%|          | 0/88 [00:00<?, ?it/s]

KeyboardInterrupt: 